# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\ShwetankPC\anaconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA=os.getenv("PRICE_DATA")
parquet_files=glob(os.path.join("../",PRICE_DATA, "*/*/*.parquet")) #additional directory level up compared to original env path (meant for materials directory)

In [4]:
parquet_files[:3]

['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
#dd_px.compute().head()

In [6]:
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532215,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206852,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712803,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276855,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299919,2819626,Health Care,Life Sciences Tools & Services,2000


In [7]:
dd_px.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')

In [10]:
# Write your code below.
import numpy as np

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] -x['Low']
))



C:\Users\ShwetankPC\AppData\Local\Temp\ipykernel_89400\988898579.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [11]:
# Write your code below.

pd_feat=dd_feat.compute()
pd_feat['Rolling Average']=pd_feat['returns'].rolling(10).mean()

In [12]:
pd_feat.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,Rolling Average
ticker,,,,,,,,,,,,,,
IP,2023-01-03,34.759998,35.509998,34.709999,35.410000,32.742538,3692600,Materials,Paper & Plastic Packaging Products & Materials,2023,NaN,NaN,0.799999,NaN
IP,2023-01-04,35.759998,36.860001,35.650002,36.830002,34.055565,4592300,Materials,Paper & Plastic Packaging Products & Materials,2023,35.410000,0.040102,1.209999,NaN
IP,2023-01-05,36.480000,36.570000,35.900002,36.200001,33.473022,3407300,Materials,Paper & Plastic Packaging Products & Materials,2023,36.830002,-0.017106,0.669998,NaN
IP,2023-01-06,36.580002,36.900002,36.349998,36.639999,33.879883,2167300,Materials,Paper & Plastic Packaging Products & Materials,2023,36.200001,0.012155,0.550003,NaN
IP,2023-01-09,36.680000,37.669998,36.680000,37.009998,34.222004,2951300,Materials,Paper & Plastic Packaging Products & Materials,2023,36.639999,0.010098,0.989998,NaN


In [13]:
pd_feat.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,Rolling Average
ticker,,,,,,,,,,,,,,
BXP,2016-12-23,124.639999,124.639999,123.019997,124.070000,90.732018,502900,Real Estate,Office REITs,2016,124.529999,-0.003694,1.620003,-0.003574
BXP,2016-12-27,124.139999,124.989998,123.300003,124.250000,90.863640,467000,Real Estate,Office REITs,2016,124.070000,0.001451,1.689995,-0.003903
BXP,2016-12-28,123.400002,123.519997,121.739998,122.459999,90.098480,509900,Real Estate,Office REITs,2016,124.250000,-0.014406,1.779999,-0.005088
BXP,2016-12-29,122.870003,124.290001,121.470001,123.669998,90.988701,497000,Real Estate,Office REITs,2016,122.459999,0.009881,2.820000,-0.001077
BXP,2016-12-30,123.820000,126.480003,123.510002,125.779999,92.541115,792500,Real Estate,Office REITs,2016,123.669998,0.017062,2.970001,0.002763


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? \
Ans: The rolling method does exist in dask so it was not.necessary

+ Would it have been better to do it in Dask? Why? \
Ans: It would have probably been better to do it in Dask as it is meant to work with the large amount of data present and would be more efficient (also saves time on conversion to pandas)

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.